In [1]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
from scipy.stats import norm
import math
from time import time as TIME

In [2]:
def H(theta, y, a, gamma):
    ind = (y >= theta).astype(int)
    val = 1.0 -  ind / (1-a) + 2 * gamma * theta
    return val

In [3]:
def single_asset(a_hat, mu, sigma):
    start = TIME()

    N = 100
    n = 10 ** 6
    l = 10 ** (-4)
    beta = 10 ** 8
    gamma = 10 ** (-8)
    #a_hat = 0.95

    #mu, sigma = 0, 1

    res_arr = np.zeros((N, 2)) #value of VaR and AVaR estimates

    y = norm.rvs(loc = mu, scale = sigma, size = (n, N)) #data
    ksi = norm.rvs(loc = 0, scale = 1, size = (n, N)) #noise

    theta_old = np.zeros(N)

    for i in range(n):
        theta_new = theta_old - l * H(theta_old, y[i], a_hat, gamma) + ksi[i] * (2 * l / beta) ** 0.5
        theta_old = theta_new

    V = np.mean( np.maximum( y.T - theta_new.reshape(N, -1), 0), 1) / (1-a_hat) + theta_new

    end = TIME()
    #print(end - start)
    return theta_new, V

In [7]:
VaR_arr1, CVaR_arr1 = single_asset(0.95, 0, 1)

In [127]:
VaR_arr2, CVaR_arr2 = single_asset(0.95, 1, 2)

In [128]:
VaR_arr3, CVaR_arr3 = single_asset(0.95, 3, 5)

In [129]:
VaR_arr11, CVaR_arr11 = single_asset(0.99, 0, 1)

In [130]:
VaR_arr22, CVaR_arr22 = single_asset(0.99, 1, 2)

In [131]:
VaR_arr33, CVaR_arr33 = single_asset(0.99, 3, 5)

In [132]:
print(np.mean([VaR_arr1, CVaR_arr1], 1))
print(np.std([VaR_arr1, CVaR_arr1], 1))

[1.64420406 2.06274902]
[0.01937561 0.0023006 ]


In [133]:
print(np.mean([VaR_arr2, CVaR_arr2], 1))
print(np.std([VaR_arr2, CVaR_arr2], 1))

[4.29172792 5.12561988]
[0.03420942 0.00482517]


In [134]:
print(np.mean([VaR_arr3, CVaR_arr3], 1))
print(np.std([VaR_arr3, CVaR_arr3], 1))

[11.22771291 13.31403562]
[0.04867111 0.01181514]


In [135]:
print(np.mean([VaR_arr11, CVaR_arr11], 1))
print(np.std([VaR_arr11, CVaR_arr11], 1))

[2.32984482 2.66675354]
[0.03772135 0.00476271]


In [136]:
print(np.mean([VaR_arr22, CVaR_arr22], 1))
print(np.std([VaR_arr22, CVaR_arr22], 1))

[5.66011068 6.33214605]
[0.0611838 0.0094544]


In [137]:
print(np.mean([VaR_arr33, CVaR_arr33], 1))
print(np.std([VaR_arr33, CVaR_arr33], 1))

[14.6352855  16.33115655]
[0.10794006 0.02683649]


In [157]:
weight(theta_old[1:]) @ np.array([y1[i], y2[i]])

250.47682034259043

In [158]:
np.sum(weight(theta_old[1:]) * np.array([y1[i], y2[i]]))

250.47682034259043

In [162]:
np.delete(theta_old[1:], 0)

array([0.50000324])

In [163]:
theta_old[1:]

array([0.50000486, 0.50000324])

In [190]:
np.insert(theta_old[1:], 0, 100)

array([100.        ,  -5.75730047,   5.75722078])

In [4]:
def weight(W):
    val = np.exp(W) / np.sum(np.exp(W))
    return val

def H1(theta, y, W, gamma):
    a_hat = 0.95
    indicator = 1.0 if np.sum(weight(W) * y) >= theta else 0.0 #(np.sum(Weight(W) * y) >= theta).astype(int)
    
    val1 = 1.0 - 1.0 / (1.0 - a_hat) * indicator + 2.0 * gamma * theta #number
    
    L = len(W)
    H = np.zeros(L)
    for j in range(L):
        g_hat = 0
        
        for i in range(L):
            if i == j:
                der = np.exp(W[j]) * np.sum(np.exp(np.delete(W,j))) / (np.sum(np.exp(W))) ** 2
            else:
                der = -np.exp(W[i]) * np.exp(W[j]) / (np.sum(np.exp(W))) ** 2
            g_hat += y[i] * der
            
        H[j] = 1.0 / (1.0 - a_hat) * indicator * g_hat + 2.0 * gamma * W[j]
    return np.insert(H, 0, val1)

def ref_val1(y):
    N = 100
    w = np.linspace(0, 1, N)
    res = 10 ** 8
    for i in range(N):
        w1 = w[i]
        for j in range(N):
            w2 = w[j]
            w3 = 1.0 - w1 - w2

            portfolio_LP = w1 * y[0] + w2 * y[1] + w3 * y[2]

            VaR = np.quantile(portfolio_LP, 0.95)
            CVaR = np.mean(portfolio_LP[portfolio_LP >= VaR])

            if CVaR <= res:
                res = CVaR
                res_arr = [w1, w2, w3, VaR, CVaR]
    return res_arr

def ref_val(y):
    N = 150
    w = np.linspace(0, 1, N)
    
    w1 = w.reshape(-1, 1, 1)
    w2 = w.reshape(1, -1, 1)
    w3 = 1.0 - w1 - w2
    
    portfolio_LP = w1 * y[0] + w2 * y[1] + w3 * y[2]
    
    VaR = np.quantile(portfolio_LP, 0.95, axis=2)
    CVaR = np.nanmean(np.where(portfolio_LP >= VaR.reshape(1, 1, -1), portfolio_LP, np.nan), axis=2)
    
    idx = np.nanargmin(CVaR)
    
    res_arr = [w[idx // N], w[idx % N], 1.0 - w[idx // N] - w[idx % N], VaR[0, idx], CVaR[0, idx]]
    
    return res_arr

In [5]:
def multi_assets(params):
    mu1, var1, mu2, var2, mu3, var3 = params
    start = TIME()

    n = 10 ** 6
    l = 10 ** (-4)
    beta = 10 ** 8
    gamma = 10 ** (-8)
    a_hat = 0.95
    m = 3 #assets number

    y1 = norm.rvs(loc = mu1, scale = var1**0.5, size = n) #data 1st asset
    y2 = norm.rvs(loc = mu2, scale = var2**0.5, size = n) #data 2st asset
    y3 = norm.rvs(loc = mu3, scale = var3**0.5, size = n) #data 3st asset
    y = np.array([y1, y2, y3])
    y_T = y.T
    ksi = norm.rvs(loc = 0, scale = 1, size = (n, m+1)) #noise

    theta_old = np.zeros(m+1)

    for i in range(n):
        theta_new = theta_old - l * H1(theta_old[0], y_T[i], theta_old[1:], gamma) + ksi[i] * (2 * l / beta) ** 0.5
        theta_old = theta_new

    V = np.mean(np.maximum(weight(theta_new[1:]) @ y - theta_new[0], 0)) / (1-a_hat) + theta_new[0]

    end = TIME()
    #print(end - start)
    return weight(theta_new[1:]), theta_new[0], V#, ref_val(y)#, y#, ref_val1(y)

In [13]:
start = TIME()
res = multi_assets([500, 1, 0, 10**6, 0, 10**-4])
print(res[:-1])
print(res[-1])
end = TIME()
print(end - start)

(array([6.95073828e-06, 3.01745692e-06, 9.99990032e-01]), 0.02343955384217421, 0.025679992040724478)
[0.0, 0.0, 1.0, 0.016449095894498177, 0.020637334524107206]
94.90452909469604


In [16]:
start = TIME()
res = multi_assets([500, 1, 0, 10**6, 0, 1])
print(res[:-1])
print(res[-1])
end = TIME()
print(end - start)

(array([8.64348681e-06, 1.94437794e-05, 9.99971913e-01]), 1.6647200245358602, 2.0685504923752407)
[0.0, 0.0, 1.0, 1.6473975706496953, 2.063613538535911]
93.98094010353088


In [6]:
start = TIME()
res = multi_assets([0, 10**3, 0, 1, 0, 4])
print(res[:-1])
print(res[-1])
end = TIME()
print(end - start)

(array([0.00156756, 0.79619441, 0.20223803]), 1.4472968061387834)
1.8426126284193736
48.97334027290344


In [ ]:
res = multi_assets([0, 10**3, 0, 1, 0, 4])

In [1]:
res[0]

NameError: name 'res' is not defined

In [2]:
np.sum([1,2])

NameError: name 'np' is not defined

In [47]:
sum(res[-2][:3])

1.0067114093959733

In [37]:
res[2]

1.8484435849909215

In [38]:
res[-2][-1]

4.111616324698849

In [40]:
res[-1]

3

In [41]:
y = res[-1]
w1,w2,w3 = res[-2][:3]
portfolio_LP = w1 * y[0] + w2 * y[1] + w3 * y[2]

VaR = np.quantile(portfolio_LP, 0.95)
CVaR = np.mean(portfolio_LP[portfolio_LP >= VaR])

In [42]:
CVaR

4.111616324698852

In [43]:
y = res[-1]
w1,w2,w3 = res[0]
portfolio_LP = w1 * y[0] + w2 * y[1] + w3 * y[2]

VaR = np.quantile(portfolio_LP, 0.95)
CVaR = np.mean(portfolio_LP[portfolio_LP >= VaR])

In [44]:
CVaR

1.8483135292831983

In [45]:
res[0]

array([0.0015349 , 0.79823866, 0.20022644])

In [46]:
VaR

1.472121942184292

In [ ]:
y1 = norm.rvs(loc = mu1, scale = var1**0.5, size = n) #data 1st asset
y2 = norm.rvs(loc = mu2, scale = var2**0.5, size = n) #data 2st asset
y3 = norm.rvs(loc = mu3, scale = var3**0.5, size = n) #data 3st asset